#Install the Azure ML SDK on your Azure Databricks Cluster

The `Azure Machine Learning Python SDK` is required for leveraging the experimentation, model management and model deployment capabilities of Azure Machine Learning services.

If your cluster is not already provisioned with the Azure Machine Learning Python SDK, you easily add it to your cluster by adding the following libraries. 

For reference, to use this notebook in your own Databricks environment, you will need to create libraries, using the [Create Library](https://docs.azuredatabricks.net/user-guide/libraries.html) interface in Azure Databricks, for the following and attach them to your cluster:

**azureml-sdk**
* Source: Upload Python Egg or PyPi
* PyPi Name: `azureml-sdk[databricks]`
* Select Install Library

Verify that the Azure ML SDK is installed on your cluster by running the following cell:

In [4]:
import azureml.core
azureml.core.VERSION

Out[ 1 ]: '0.1.68'

If you see a version number output in the above cell, your cluster is ready to go.

#Initialize Azure ML Workspace

In this notebook, you will use the Azure Machine Learning SDK to create a new Azure Machine Learning Workspace in your Azure Subscription.

Please specify the Azure subscription Id, resource group name, workspace name, and the region in which you want to create the Azure Machine Learning Workspace. 

You can get the value of your Azure subscription ID from the Azure Portal, and then selecting Subscriptions from the menu on the left.

For the `resource_group`, use the name of the resource group that contains your Azure Databricks Workspace. 

NOTE: If you provide a resource group name that does not exist, the resource group will be automatically created. This may or may not succeed in your environment, depending on the permissions you have on your Azure Subscription.

In [7]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = "e223f1b3-d19b-4cfa-98e9-bc9be62717bc"#"<you-azure-subscription-id>"

#Provide a name for the new Resource Group that will contain Azure ML related services 
resource_group = "AI-Practice-Dev-EastUS"#"<resource-group-name>"

# Proivde the name and region for the Azure Machine Learning Workspace that will be created
workspace_name = "azure-ml-ws-zst"#"<azure-ml-workspace-name>"
workspace_region = "eastus2"#'eastus2' # eastus, westcentralus, southeastasia, australiaeast, westeurope

#Create an Azure ML Workspace

Run the following cell and follow the instructions printed in the output. 

You will see instructions that read:

`Performing interactive authentication. Please follow the instructions on the terminal.`

`To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code SOMECODE to authenticate.`

When you see this, open a new browser window, navigate to the provided URL. At the code prompt, enter the code provided (be sure to delete any trailing spaces).

Login with the same credentials you use to access your Azure subscription.

Once you have authenticated, the output will continue.

When you see `Provisioning complete.` your Workspace has been created and you can move on to the next cell.

In [10]:
import azureml.core

# import the Workspace class and check the azureml SDK version
from azureml.core import Workspace

ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region)

print("Provisioning complete.")

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code B6HWJ8TV6 to authenticate.
Interactive authentication successfully completed.
Provisioning complete.

#Persist the Workspace configuration

Run the following cells to retrieve the configuration of the deployed Workspace and persist it to local disk and then to the Databricks Filesystem.

In [13]:
import os
import shutil
import azureml.core
# import the Workspace class and check the azureml SDK version
from azureml.core import Workspace

ws = Workspace(
    workspace_name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group)

# persist the subscription id, resource group name, and workspace name in aml_config/config.json.
aml_config = 'aml_config'
if os.path.isfile(aml_config) or os.path.isdir(aml_config):
    shutil.rmtree(aml_config)
ws.write_config()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BMNRLMTNX to authenticate.
Interactive authentication successfully completed.
Wrote the config file config.json to: /databricks/driver/aml_config/config.json

Take a look at the contents of the generated configuration file by running the following cell:

In [15]:
%sh
cat /databricks/driver/aml_config/config.json

{
 "resource_group": "AI-Practice-Dev-EastUS",
 "subscription_id": "e223f1b3-d19b-4cfa-98e9-bc9be62717bc",
 "workspace_name": "azure-ml-ws-zst"
}

Copy the config file to DBFS

In [17]:
#persist the config file to dbfs so that it can be used for the other notebooks.
aml_config_local = 'file:' + os.getcwd() + '/' + aml_config
aml_config_dbfs = '/dbfs/' + 'aml_config'

if os.path.isfile(aml_config_dbfs) or os.path.isdir(aml_config_dbfs):
    shutil.rmtree(aml_config_dbfs)
    #dbutils.fs.rm(aml_config, recurse=True)

dbutils.fs.cp(aml_config_local, aml_config, recurse=True)

Out[ 6 ]: True

#Deploy model to Azure Container Instance (ACI)

In this notebook, you will deploy the best performing model you selected previously as a web service hosted in Azure Container Service.

## Copy the model from DBFS

You previously saved the model in DBFS, but to deploy it using Azure Machine Learning services, you will need to copy the model to local storage on the driver node.

Run the following cells to copy the model from DBFS to local and verify that you can load the model.

In [22]:
##NOTE: service deployment always gets the model from the current working dir. 
tempFolderName = '/FileStore/ignite2018_05_03_{0}'.format("8fbaedaa-07f8-4259-85e5-19c4c122a869") # UUID from previous notebook
fileName1 = '{0}/claim_classifier.tfl.data-00000-of-00001'.format(tempFolderName)
fileName2 = '{0}/claim_classifier.tfl.index'.format(tempFolderName)
fileName3 = '{0}/claim_classifier.tfl.meta'.format(tempFolderName)

model_name = "claim_classifier.tfl"
model_path_dbfs = fileName1 #os.path.join("/dbfs/models", model_name)
model_path_local = "file:" + os.getcwd() + "/" + model_name + "/"

print("copy model from dbfs {} to under local folder {}".format(model_path_dbfs, model_path_local))
os.mkdir("/databricks/driver/claim_classifier.tfl")
dbutils.fs.cp(fileName1, model_path_local, recurse=False)
dbutils.fs.cp(fileName2, model_path_local, recurse=False)
dbutils.fs.cp(fileName3, model_path_local, recurse=False)

copy model from dbfs /FileStore/ignite2018_05_03_8fbaedaa-07f8-4259-85e5-19c4c122a869/claim_classifier.tfl.data-00000-of-00001 to under local folder file:/databricks/driver/claim_classifier.tfl/
 Out[ 19 ]: True

In [23]:
os.listdir("/databricks/driver/claim_classifier.tfl")

Out[ 20 ]: 
['claim_classifier.tfl.data-00000-of-00001',
 '.claim_classifier.tfl.data-00000-of-00001.crc',
 '.claim_classifier.tfl.index.crc',
 '.claim_classifier.tfl.meta.crc',
 'claim_classifier.tfl.index',
 'claim_classifier.tfl.meta']

# Register the model with Azure Machine Learning

Begin by loading your Azure Machine Learning Workspace configuration from disk.

In [26]:
import azureml.core
from azureml.core.workspace import Workspace

#get the config file from dbfs
aml_config = '/aml_config'
dbutils.fs.cp(aml_config, 'file:'+os.getcwd()+aml_config, recurse=True)

ws = Workspace.from_config()

Found the config file in: /databricks/driver/aml_config/config.json

In the following, you register the model file with Azure Machine Learning (which saves a copy of the model in the cloud).

In [28]:
#Register the model
from azureml.core.model import Model
mymodel = Model.register(model_path = model_name + "/", # this points to a local file or folder in the current working dir
                       model_name = model_name, # this is the name the model is registered with                 
                       description = "Claims classification model.",
                       workspace = ws)

print(mymodel.name, mymodel.description, mymodel.version)

Registering model claim_classifier.tfl
claim_classifier.tfl Claims classification model. 4

#Create the scoring web service

When deploying models for scoring with Azure Machine Learning services, you need to define the code for a simple web service that will load your model and use it for scoring. By convention this service has two methods `init` which loads the model and `run` which scores data using the loaded model. 

This scoring service code will later be deployed inside of a specially prepared Docker container.

In [31]:
#%%writefile score_classifer.py
score_classifier = """

import numpy as np
import re
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical
import nltk
import uuid
import json
import os
import sys

def init():
    try:
        print("init() called.")
        
        global model2
        global net2
        global vectorizer
        
        # Takes at most a couple of minutes to download all NLTK content
        print("downloading nltk.")
        nltk.download("all")
        
        tempFolderName = './resources'
        os.mkdir(tempFolderName)
        print('Content files will be saved to {0}'.format(tempFolderName))

        print("downloading files...")
        filesToDownload = ['claims_labels.txt', 'claims_text.txt', 'contractions.py', 'textanalytics.py']
        for fileToDownload in filesToDownload:
          downloadCommand = 'wget -O ''{0}/{1}'' ''https://databricksdemostore.blob.core.windows.net/data/05.03/{1}'''.format(tempFolderName, fileToDownload)
          print(downloadCommand)
          os.system(downloadCommand)
        print("file download complete.")
        
        print("importing textanalytics...")
        sys.path.append(os.path.abspath('./{0}'.format(tempFolderName)))      
        import textanalytics as ta
        print("importing succeeded.")
        
        
        claims_corpus = [claim for claim in open("claims_text.txt")]

        norm_corpus = ta.normalize_corpus(claims_corpus)
        vectorizer, tfidf_matrix = ta.build_feature_matrix(norm_corpus) 
        
        print("creating network")
        # Build the neural network and then load its weights from disk
        net2 = tflearn.input_data(shape=[None, 258])
        net2 = tflearn.fully_connected(net2, 32)
        net2 = tflearn.fully_connected(net2, 32)
        net2 = tflearn.fully_connected(net2, 2, activation='softmax')
        net2 = tflearn.regression(net2)
        
        print("loading model from disk...")
        from azureml.core.model import Model
        model2 = tflearn.DNN(net2)
        model_name = "claim_classifier.tfl" 
        model_path = Model.get_model_path(model_name)
        model2.load(model_path, weights_only=True)
        print("model loaded.")
        
        print("init complete.")

    except Exception as e:
        print("Exception in init: " + str(e))
        trainedModel = e

def run(input_df):
    response = ''    

    try:
        print("calling run with: " + input_df)     
        
        
        print("importing textanalytics...")
        import textanalytics as ta
        
        print("using hard coded values")
        test_claim = ['I crashed my car into a pole.', 'The flood ruined my house.', 'I lost control of my car and fell in the river.']
        test_claim = ta.normalize_corpus(test_claim)
        test_claim = vectorizer.transform(test_claim)
        test_claim = test_claim.toarray()
        
        print("calling model2.predict_label")
        preds = model2.predict_label(test_claim)
        
        response = json.dumps(preds)
        
        print("response: " + str(response))
        
    except Exception as e:
        print("Exception in run: " + str(e))
        return (str(e))

    # Return results
    return response
    
"""

exec(score_classifier)

with open("score_classifier.py", "w") as file:
    file.write(score_classifier)

In [32]:
import numpy as np
import re
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical
import nltk
import uuid
import json
import os
import sys

def init():
    try:
        print("init() called.")
        
        global model2
        global net2
        global vectorizer
        
        # Takes at most a couple of minutes to download all NLTK content
        print("downloading nltk.")
        nltk.download("all")
        
        tempFolderName = './resources'
        os.mkdir(tempFolderName)
        print('Content files will be saved to {0}'.format(tempFolderName))

        print("downloading files...")
        filesToDownload = ['claims_labels.txt', 'claims_text.txt', 'contractions.py', 'textanalytics.py']
        for fileToDownload in filesToDownload:
          downloadCommand = 'wget -O ''{0}/{1}'' ''https://databricksdemostore.blob.core.windows.net/data/05.03/{1}'''.format(tempFolderName, fileToDownload)
          print(downloadCommand)
          os.system(downloadCommand)
        print("file download complete.")
        
        print("importing textanalytics...")
        sys.path.append(os.path.abspath('./{0}'.format(tempFolderName)))      
        import textanalytics as ta
        print("importing succeeded.")
        
        
        claims_corpus = [claim for claim in open("claims_text.txt")]

        norm_corpus = ta.normalize_corpus(claims_corpus)
        vectorizer, tfidf_matrix = ta.build_feature_matrix(norm_corpus) 
        
        print("creating network")
        # Build the neural network and then load its weights from disk
        net2 = tflearn.input_data(shape=[None, 258])
        net2 = tflearn.fully_connected(net2, 32)
        net2 = tflearn.fully_connected(net2, 32)
        net2 = tflearn.fully_connected(net2, 2, activation='softmax')
        net2 = tflearn.regression(net2)
        
        print("loading model from disk...")
        from azureml.core.model import Model
        model2 = tflearn.DNN(net2)
        model_name = "claim_classifier.tfl" 
        model_path = Model.get_model_path(model_name)
        model2.load(model_path, weights_only=True)
        print("model loaded.")
        
        print("init complete.")

    except Exception as e:
        print("Exception in init: " + str(e))
        trainedModel = e

def run(input_df):
    response = ''    

    try:
        print("calling run with: " + input_df)     
        
        
        print("importing textanalytics...")
        import textanalytics as ta
        
        print("using hard coded values")
        test_claim = ['I crashed my car into a pole.', 'The flood ruined my house.', 'I lost control of my car and fell in the river.']
        test_claim = ta.normalize_corpus(test_claim)
        test_claim = vectorizer.transform(test_claim)
        test_claim = test_claim.toarray()
        
        print("calling model2.predict_label")
        preds = model2.predict_label(test_claim)
        
        response = json.dumps(preds)
        
        print("response: " + str(response))
        
    except Exception as e:
        print("Exception in run: " + str(e))
        return (str(e))

    # Return results
    return response

# Create a Conda dependencies environment file

Your scoring service can have dependencies install by using a Conda environment file. Items listed in this file will be conda or pip installed within the Docker container that is created and thus be available to your scoring web service logic.

In [35]:
from azureml.core.conda_dependencies import CondaDependencies 

myacienv = CondaDependencies.create(conda_packages=['scikit-learn','numpy','pandas'], pip_packages=['nltk','tensorflow','tflearn'])

with open("mydeployenv.yml","w") as f:
    f.write(myacienv.serialize_to_string())

#Deployment

In the following cells you will use the Azure Machine Learning SDK to package the model and scoring script in a container, and deploy that container to an Azure Container Instance.

Run the following cells.

Create a configuration of the ACI web service instance that provides the number of CPU cores, size of memory, a collection of tags and a description.

In [39]:
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    tags = {'name':'Claim Classification'}, 
    description = 'Classifies a claim as home or auto.')

Next, build up a container image configuration that names the scoring service script, the runtime (python or Spark), and provides the conda file.

In [41]:
service_name = "claimclassservice10"
runtime = "python" #"python" #
driver_file = "score_classifier.py"
conda_file = "mydeployenv.yml"

from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = driver_file,
                                                  runtime = runtime,
                                                  conda_file = conda_file)

Now you are ready to begin your deployment to the Azure Container Instance. 

Run the following cell. This may take between **5-15 minutes** to complete.

You will see output similar to the following when your web service is ready:
`SucceededACI service creation operation finished, operation "Succeeded"`

In [43]:
webservice = Webservice.deploy_from_model(
  workspace=ws, 
  name=service_name, 
  deployment_config=aci_config,
  models = [mymodel], 
  image_config=image_config, 
  )

webservice.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image claimclassservice10:1, operation "Succeeded"
Creating service
Running............................................
SucceededACI service creation operation finished, operation "Succeeded"

#Test the deployed service

Now you are ready to test scoring using the deployed web service. The following cell invokes the web service. 

Run the following cells to test scoring using a single input row against the deployed web service.

In [46]:
test_claim = ['I crashed my car into a pole.', 'The flood ruined my house.', 'I lost control of my car and fell in the river.']

In [47]:
import json
json_str_test_inputs = json.dumps(test_claim)
webservice.run(input_data = json_str_test_inputs)

Out[ 35 ]: "\n**********************************************************************\n Resource 'corpora/stopwords' not found. Please use the NLTK\n Downloader to obtain the resource: >>> nltk.download()\n Searched in:\n - '/home/mmlspark/nltk_data'\n - '/usr/share/nltk_data'\n - '/usr/local/share/nltk_data'\n - '/usr/lib/nltk_data'\n - '/usr/local/lib/nltk_data'\n**********************************************************************"

#Clean up

When you are finished experimenting with your deployed web service, you can also use the Azure Machine Learning Python SDK to delete the deployed service.

Run the following cell to cleanup.

In [50]:
webservice.delete()

# You are done!

Congratulations, you have completed this team challenge!